In [14]:
import pandas as pd
import numpy as np
import json
import os
from glob import glob

## Read in Table

In [15]:
# path: str = "/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Bräsen/RCC-Masterfile_20211124.xlsx" "A,N,O,AH,GQ,GR,GS,K"
path: str = "/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Braesen/RCC-Masterfile 2023-02-22.xlsx"

scan_path:str = "/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Braesen/"

lut = {
    'A':"Scan-Code",
    'K':'IO_PFS_Months', 
    'N':'Follow Up/Todesdatum Stand 13.04.2022',
    'O':'Status 13.04.2022', 
    'AK':'Nephrektomiedatum', 
    'BV':'Neue Klassifikation',
    'GT': 'Status 0=lebt 1=Tod', 
    'GU': 'FollowUp_Todesdatum', 
    'GV': 'Datum_ED'
    }


raw_data = pd.read_excel(
    path, 
    sheet_name="Proben RCC, PD-L1, Metastas (2", 
    header=0,
    index_col=0,
    nrows=178,
    usecols=','.join(lut.keys())
    )
raw_data = raw_data[raw_data.index.notnull()]
# add excel index column indices 
raw_data.columns = pd.MultiIndex.from_tuples(zip(*[{key: value for key, value in lut.items() if key != 'A'}.keys(), raw_data.columns]))#  [letter for letter, col in zip(["A","N","O","GQ","GR","GS","K"], data.columns)]#
# use only case id
raw_data.index = [case[-3:]# int(case[-3:])
 for case in raw_data.index.astype(str)]
raw_data.columns

MultiIndex([( 'K',                         'IO_PFS_Months'),
            ( 'N', 'Follow Up/Todesdatum Stand 13.04.2022'),
            ( 'O',                     'Status 13.04.2022'),
            ('AK',                     'Nephrektomiedatum'),
            ('BV',                   'Neue Klassifikation'),
            ('GT',                   'Status 0=lebt 1=Tod'),
            ('GU',                   'FollowUp_Todesdatum'),
            ('GV',                              'Datum_ED')],
           )

In [16]:
# now column letters are aliases to the tuple of column letter and name
_g = globals()
for char in lut.keys():
    _g["_"+char] = (char ,lut[char])

In [17]:
raw_data[_BV].value_counts()

ccRCC                                           133
ccRCC (sarkomatoid)                              12
ccRCC (rhabdoid)                                  6
chromoph. RCC                                     5
NOS                                               5
ccRCC (sarkomatoid, rhabdoid)                     5
pRCC                                              3
pRCC Typ II                                       2
ccRCC; medullär mgl.                              1
ccRCC (eosinophil)                                1
pRCC Typ II (sarkomatoid)                         1
pRCC (eosinophil, "special type")                 1
ccRCC (sarkomatoid), partiell SDHB (schwach)      1
Name: (BV, Neue Klassifikation), dtype: int64

In [18]:
raw_data

,K,N,O,AK,BV,GT,GU,GV
,IO_PFS_Months,Follow Up/Todesdatum Stand 13.04.2022,Status 13.04.2022,Nephrektomiedatum,Neue Klassifikation,Status 0=lebt 1=Tod,FollowUp_Todesdatum,Datum_ED
063,NaN,NaT,NaN,NaT,ccRCC,1.0,2009-03-15,1987-06-15
067,NaN,NaT,NaN,NaT,ccRCC,NaN,NaT,NaT
062,NaN,NaT,NaN,2001-01-24,ccRCC,1.0,2015-05-15,2001-01-24
068,NaN,NaT,NaN,2001-11-27,ccRCC,1.0,2010-08-09,2001-11-27
105,NaN,NaT,NaN,2001-06-13,pRCC Typ II,0.0,2016-05-31,2001-06-13
...,...,...,...,...,...,...,...,...
191,4.200000,2021-11-23,lebt,2011-05-12,ccRCC,0.0,2021-11-23,2011-05-12
192,1.900000,2021-12-07,lebt,2011-11-14,ccRCC,0.0,2021-12-07,2011-11-01
193,2.833333,2021-12-07,tot,2008-03-12,ccRCC,1.0,2021-12-07,2008-03-12


## Create censoring status


In [19]:
# copy data
data = raw_data.copy()
# replace values with 0,1
data = data.replace({_O :{'lebt': 0, 'tot': 1}})

# Rows where the date is the same but the status does not agree
test = data[data[_N] == raw_data[_GU]]
(test[test[_O] != test[_GT]].sort_index())

,K,N,O,AK,BV,GT,GU,GV
,IO_PFS_Months,Follow Up/Todesdatum Stand 13.04.2022,Status 13.04.2022,Nephrektomiedatum,Neue Klassifikation,Status 0=lebt 1=Tod,FollowUp_Todesdatum,Datum_ED
119,2.400000,2021-03-05,1.0,2013-11-05,ccRCC,0.0,2021-03-05,2013-11-12
129,2.600000,2016-04-28,1.0,2015-03-11,ccRCC (sarkomatoid),0.0,2016-04-28,2015-03-11
130,1.800000,2018-03-26,1.0,2014-05-26,ccRCC,0.0,2018-03-26,2014-05-15
136,5.966667,2021-12-21,0.0,2011-09-14,ccRCC,1.0,2021-12-21,2011-09-14
161,1.766667,2022-01-06,0.0,2016-03-01,"ccRCC (sarkomatoid, rhabdoid)",1.0,2022-01-06,2016-02-17
176,7.100000,2021-02-01,1.0,2020-05-12,ccRCC (sarkomatoid),0.0,2021-02-01,2020-05-12



censored if:
- a patient has not (yet) experienced the event of interest, such as relapse or death, within the study time period;
- a patient is lost to follow-up during the study period;
- a patient experiences a different event that makes further follow-up impossible.


In [20]:
# step 1: get date & status from 13.04.22 if available else fall back to other column
data["date"] = data[_N].combine_first(data[_GU])
data["death"] = data[_O].combine_first(data[_GT])

# step 2: get censoring status, not dead on 04.04.22 
data["uncensored"] = (data.loc[:,"date"] > pd.Series(['2022-04-04'] * len(data), name="date", index=data.index)).astype(float) + data.loc[:,"death"]
data['surv_days'] = (data['date'] - data[_AK]).dt.days

In [21]:
data.sort_values(by=[_BV], ascending=False)

,K,N,O,AK,BV,GT,GU,GV,date,death,uncensored,surv_days
,IO_PFS_Months,Follow Up/Todesdatum Stand 13.04.2022,Status 13.04.2022,Nephrektomiedatum,Neue Klassifikation,Status 0=lebt 1=Tod,FollowUp_Todesdatum,Datum_ED,,,,
106,NaN,NaT,NaN,2008-04-23,pRCC Typ II (sarkomatoid),1.0,2008-10-14,2008-03-11,2008-10-14,1.0,1.0,174.0
157,33.933333,2021-11-30,0.0,2013-01-23,pRCC Typ II,0.0,2021-11-30,2013-01-23,2021-11-30,0.0,0.0,3233.0
105,NaN,NaT,NaN,2001-06-13,pRCC Typ II,0.0,2016-05-31,2001-06-13,2016-05-31,0.0,0.0,5466.0
098,NaN,NaT,NaN,2010-04-26,"pRCC (eosinophil, ""special type"")",1.0,2011-09-06,2010-04-26,2011-09-06,1.0,1.0,498.0
194,2.233333,2019-01-09,0.0,2018-05-02,pRCC,0.0,2019-01-09,2018-05-02,2019-01-09,0.0,0.0,252.0
...,...,...,...,...,...,...,...,...,...,...,...,...
154,NaN,NaT,NaN,2011-01-05,NOS,1.0,2011-05-27,2010-12-22,2011-05-27,1.0,1.0,142.0
185,1.366667,2020-04-24,0.0,2012-05-01,NOS,0.0,2020-04-24,2012-05-01,2020-04-24,0.0,0.0,2915.0
178,2.333333,2019-11-11,0.0,2018-11-28,NOS,0.0,2019-11-11,2018-11-09,2019-11-11,0.0,0.0,348.0


## Filtering

read cases that appear in qupath projects

In [22]:
# recursively find all .qpproj files in a directory
gproj_list: list[str] = glob(os.path.join(scan_path, "**", "*.qpproj"), recursive=True)
[(it.split("/", maxsplit=6))[-1] for it in gproj_list]

['RCC-MaskenБbertragung CD4_FoxP3 N. Vasileiadis - Kopie/project.qpproj',
 'RCC-MaskenБbertragung CD4_FoxP3 N. Vasileiadis - Kopie/project.backup.qpproj',
 'hand-editted/RCC-Maskenubertragung-CD8_CD20-N.Vasileiadis/project.qpproj',
 'hand-editted/RCC-Maskenubertragung-CD4_FoxP3-N.Vasileiadis/project.qpproj',
 'hand-editted/RCC-Maskenubertragung-CD204-N.Vasileiadis/project.qpproj',
 'hand-editted/RCC-Maskenubertragung-CD68-N.Vasileiadis/project.qpproj',
 'RCC-MaskenБbertragung CD68 N. Vasileiadis - Kopie/project (2).backup.qpproj',
 'RCC-MaskenБbertragung CD68 N. Vasileiadis - Kopie/project.qpproj',
 'RCC-MaskenБbertragung CD68 N. Vasileiadis - Kopie/project.backup.qpproj',
 'RCC-MaskenБbertragung CD8_CD20 N. Vasileiadis - Kopie/project.qpproj',
 'RCC-MaskenБbertragung CD8_CD20 N. Vasileiadis - Kopie/project.backup.qpproj',
 'RCC-MaskenБbertragung CD204 N. Vasileiadis - Kopie/project.qpproj',
 'RCC-MaskenБbertragung CD204 N. Vasileiadis - Kopie/project.backup.qpproj']

In [23]:
gproj_dicts = [json.load(open(i)) for i in gproj_list]
idx_i = 1
print(gproj_dicts[idx_i]["uri"])

svs_files: list[str] = [item['serverBuilder']["uri"] for idx in range(len(gproj_dicts)) for item in gproj_dicts[idx]["images"]]
# svs_files: list[str] = [item['serverBuilder']["uri"] for item in gproj_dicts[idx_i]["images"]]

# use set to remove duplicates
svs_cases = set(os.path.basename(it).removesuffix(".svs") for it in svs_files)

file:/P:/Pathologie/AG_Nephropathologie/Vasileiadis/QuPathProjekt%20RCC%20(PräDikNika)/RCC-Maskenübertragung/RCC-Maskenübertragung%20CD4_FoxP3%20N.%20Vasileiadis/project.qpproj


In [24]:
print(len(set(svs_cases)))
svs_cases

774


{'M1-101.016~B',
 'M1-101.019~E',
 'M1-113.016~B',
 'M1-113.019~B',
 'M1-119.016~B',
 'M1-119.019~B',
 'M1-136.016~B',
 'M1-136.019~B',
 'M1-151.016~B',
 'M1-151.019~B',
 'M1-156.016~B',
 'M1-156.019~B',
 'M1-159.016~B',
 'M1-159.019~B',
 'M1-161.016flipped',
 'M1-161.019~C',
 'M1-162.016~B',
 'M1-162.019~B',
 'M1-163.016~B',
 'M1-163.019~B',
 'M1-165.016~B',
 'M1-165.019~B',
 'M1-166.016~B',
 'M1-166.019~B',
 'M1-167.016~B',
 'M1-167.019~B',
 'M1-173.016~B',
 'M1-173.019~B',
 'M2-136.016~B',
 'M2-136.019~B',
 'M2-151.016~B',
 'M2-151.019~B',
 'M2-162.016~C',
 'M2-162.019~B',
 'M2-173.016~B',
 'M2-173.019~B',
 'M3-173.016~B',
 'M3-173.019~B',
 'RCC-TA-001.016~B',
 'RCC-TA-001.019~E',
 'RCC-TA-001.041~B',
 'RCC-TA-001.042~B',
 'RCC-TA-002.016~B',
 'RCC-TA-002.019~E',
 'RCC-TA-002.041~B',
 'RCC-TA-002.042~B',
 'RCC-TA-003.016~B',
 'RCC-TA-003.019~B',
 'RCC-TA-003.041~B',
 'RCC-TA-003.042~B',
 'RCC-TA-004.016~B',
 'RCC-TA-004.019~E',
 'RCC-TA-004.041~B',
 'RCC-TA-004.042~B',
 'RCC-TA-005.

Filter data based on cases in qupath projects, also drop rows with no values

GS, the rquired column, has no nan values. The other columns will be used to piece together death/censoring times

In [25]:
qupath_cases: list[str] = [case.split(".")[0][-3:] for case in svs_cases]
data: pd.DataFrame = data[data.index.isin(qupath_cases)]
data = data.dropna(how='all')
data.sort_values(by=['surv_days'], ascending=False)

,K,N,O,AK,BV,GT,GU,GV,date,death,uncensored,surv_days
,IO_PFS_Months,Follow Up/Todesdatum Stand 13.04.2022,Status 13.04.2022,Nephrektomiedatum,Neue Klassifikation,Status 0=lebt 1=Tod,FollowUp_Todesdatum,Datum_ED,,,,
149,6.900000,2019-08-28,1.0,1995-10-18,ccRCC,0.0,2016-05-31,1995-10-18,2019-08-28,1.0,1.0,8715.0
151,1.266667,2020-08-11,1.0,1999-10-22,ccRCC,0.0,2016-05-11,1999-10-22,2020-08-11,1.0,1.0,7599.0
135,NaN,NaT,NaN,1992-02-29,ccRCC,1.0,2010-08-20,1992-07-15,2010-08-20,1.0,1.0,6747.0
042,NaN,NaT,NaN,1995-05-22,ccRCC,1.0,2013-04-29,1995-05-22,2013-04-29,1.0,1.0,6552.0
064,7.800000,2018-08-29,0.0,2001-04-12,ccRCC,0.0,2018-08-29,2001-04-12,2018-08-29,0.0,0.0,6348.0
...,...,...,...,...,...,...,...,...,...,...,...,...
150,NaN,NaT,NaN,1996-08-01,ccRCC,NaN,NaT,NaT,NaT,NaN,NaN,NaN
152,NaN,NaT,NaN,1999-03-09,ccRCC,NaN,NaT,NaT,NaT,NaN,NaN,NaN
155,NaN,NaT,NaN,2012-02-28,ccRCC (rhabdoid),NaN,NaT,NaT,NaT,NaN,NaN,NaN


In [26]:
data = data[data.loc[:, _BV].apply(lambda tum_type: tum_type.startswith('ccRCC'))]
len(data)

155

In [27]:
data = data.dropna(how='any', subset=[_AK, ('surv_days', ""), ('death', "")])
len(data)

141

## Write Subset to File

In [28]:
final_data = data.loc[:, ['surv_days', "death", "uncensored",]]
final_data = final_data.sort_index().reset_index(names=["case"])
final_data.columns = final_data.columns.get_level_values(0) # drop leftover multi-column index

In [29]:
final_data

,case,surv_days,death,uncensored
0,001,218.0,1.0,1.0
1,002,2466.0,1.0,1.0
2,003,550.0,1.0,1.0
3,004,331.0,1.0,1.0
4,005,27.0,1.0,1.0
...,...,...,...,...
136,184,2490.0,0.0,0.0
137,186,2103.0,0.0,0.0
138,187,2023.0,0.0,0.0
139,188,961.0,0.0,0.0


We have exactly one tie of survival times in our final data

In [37]:
final_data[['surv_days', 'death']].value_counts()

surv_days  death
1183.0     1.0      2
2081.0     1.0      1
1729.0     1.0      1
1792.0     0.0      1
1828.0     0.0      1
                   ..
545.0      1.0      1
539.0      0.0      1
500.0      1.0      1
490.0      1.0      1
8715.0     1.0      1
Length: 140, dtype: int64

In [38]:
final_data[final_data['surv_days'] == 1183.0 ]


,case,surv_days,death,uncensored
75,093,1183.0,1.0,1.0
83,114,1183.0,1.0,1.0


In [41]:
raw_data.loc[['093', '114']]

,K,N,O,AK,BV,GT,GU,GV
,IO_PFS_Months,Follow Up/Todesdatum Stand 13.04.2022,Status 13.04.2022,Nephrektomiedatum,Neue Klassifikation,Status 0=lebt 1=Tod,FollowUp_Todesdatum,Datum_ED
093,NaN,NaT,NaN,2010-08-04,ccRCC,1.0,2013-10-30,2010-08-04
114,NaN,NaT,NaN,2012-08-28,ccRCC,1.0,2015-11-24,2012-08-28


In [44]:
final_data.to_csv(scan_path+"survival_status.csv", index=False, lineterminator='\n')


## Various checks

Todo:
    - check if death dates differ, if not merge
    - calculate difference between death and "ED"
    - check if calculated survival aligns with PFS_Months

Check if death dates are identical

In [45]:
death_22 = data.iloc[:,1]
death_else = data.iloc[:,4]
for idx in range(len(data)):
    if not pd.isnull(death_22[idx]) and not pd.isnull(death_else[idx]):
        print(f"{idx}: {death_22[idx] == death_else[idx]} | case {data.iloc[idx].name} \n death22 \t {death_22[idx]} \n death_else \t {death_else[idx]}")

2: False | case 064 
 death22 	 2018-08-29 00:00:00 
 death_else 	 ccRCC
26: False | case 074 
 death22 	 2017-11-27 00:00:00 
 death_else 	 ccRCC
78: False | case 147 
 death22 	 2018-09-18 00:00:00 
 death_else 	 ccRCC
83: False | case 111 
 death22 	 2021-05-18 00:00:00 
 death_else 	 ccRCC
84: False | case 136 
 death22 	 2021-12-21 00:00:00 
 death_else 	 ccRCC
88: False | case 113 
 death22 	 2018-12-29 00:00:00 
 death_else 	 ccRCC
92: False | case 145 
 death22 	 2016-02-23 00:00:00 
 death_else 	 ccRCC
95: False | case 121 
 death22 	 2016-08-17 00:00:00 
 death_else 	 ccRCC
96: False | case 119 
 death22 	 2021-03-05 00:00:00 
 death_else 	 ccRCC
101: False | case 129 
 death22 	 2016-04-28 00:00:00 
 death_else 	 ccRCC (sarkomatoid)
102: False | case 128 
 death22 	 2018-03-13 00:00:00 
 death_else 	 ccRCC
103: False | case 130 
 death22 	 2018-03-26 00:00:00 
 death_else 	 ccRCC
105: False | case 146 
 death22 	 2022-01-26 00:00:00 
 death_else 	 ccRCC (sarkomatoid, rhabdoi

Not all death dates are the same, how to deal with that??

Difference between death and ED

In [46]:
diagnosis_date = data.iloc[:,-1]

In [47]:
data[diagnosis_date.isnull()]


,K,N,O,AK,BV,GT,GU,GV,date,death,uncensored,surv_days
,IO_PFS_Months,Follow Up/Todesdatum Stand 13.04.2022,Status 13.04.2022,Nephrektomiedatum,Neue Klassifikation,Status 0=lebt 1=Tod,FollowUp_Todesdatum,Datum_ED,,,,


In [48]:
surv_22 = (death_22 - diagnosis_date).dropna()
surv_else = (death_else - diagnosis_date).dropna()

UFuncTypeError: ufunc 'subtract' cannot use operands with types dtype('<M8[ns]') and dtype('float64')

These patients have no data?

In [ ]:
data[diagnosis_date.isnull()].index

Index(['067', '066', '077', '137', '134', '150', '152', '155', '168', '196'], dtype='object')

Compare death times to PFS

In [ ]:
pfs = data.iloc[:,0].dropna()
pfs.head()

064     7.800000
141    10.433333
101     2.300000
074     6.866667
147     8.400000
Name: (A, IO_PFS_Months), dtype: float64

In [ ]:
(((death_else - diagnosis_date) / np.timedelta64(1, 'M'))  - data.iloc[:,0]).dropna()

064    200.762804
141    165.307442
101    147.616836
074    146.894191
147     86.714889
111    -70.992224
136    117.272004
113     39.528902
145     28.194117
121     27.860309
119     85.322541
125      4.621056
129     11.001922
128     47.808807
130     44.558241
146      4.462547
149    240.530132
151    197.373963
156    107.796717
157     72.286507
159     13.998893
160     57.645652
161     68.871334
162     33.256336
163     16.805912
164     19.782950
165     18.143713
166     36.035272
167      7.369566
169     12.509744
170      9.514520
171      5.278977
172    120.660866
173     62.075507
174     35.707372
175     17.857751
176      1.606544
177     10.566675
178      9.724409
181     56.720931
184     89.700694
185     94.405320
186     26.580903
187      4.632097
188      1.588217
189    116.841062
190     63.224600
191    122.225594
192    119.301667
193    162.032475
194      6.046097
dtype: float64

In [ ]:
(((death_22 - diagnosis_date) / np.timedelta64(1, 'M'))  - data.iloc[:,0]).dropna()

064    200.762804
141    165.307442
101    147.616836
074    146.894191
147     86.714889
111     -6.366667
136    117.272004
113     70.478203
145     28.194117
121     30.422990
119     85.322541
125      8.925046
129     11.001922
128     47.808807
130     44.558241
146      4.462547
149    279.430315
151    248.397597
156    107.796717
157     72.286507
159     13.998893
160     57.645652
161     68.871334
162     33.256336
163     16.805912
164     19.782950
165     17.125212
166     36.035272
167      6.351065
169     12.509744
170      9.514520
171      5.278977
172    120.660866
173     62.075507
174     35.707372
175     17.857751
176      1.606544
177     10.566675
178      9.724409
181     56.720931
184     89.700694
185     94.405320
186     26.580903
187      4.632097
188      1.588217
189    116.841062
190     63.224600
191    122.225594
192    119.301667
193    162.032475
194      6.046097
dtype: float64